In [1]:
import pandas as pd

In [23]:
#1 import data
path = "your_own_path_here/titanic.csv"
titanic = pd.read_csv(path)

In [24]:
titanic.head()

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [4]:
# Filter feature and target values
x = titanic[["pclass", "age", "sex"]]
y = titanic["survived"]

In [5]:
x.head()

,pclass,age,sex
0,1st,29.0000,female
1,1st,2.0000,female
2,1st,30.0000,male
3,1st,25.0000,female
4,1st,0.9167,male


In [6]:
y.head()

0    1
1    0
2    0
3    0
4    1
Name: survived, dtype: int64

In [7]:
#2 Data preprocessing
#1） Fill NaN values with mean
x["age"].fillna(x["age"].mean(), inplace = True)

/Users/liusitong/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [9]:
#2) Turn x into Dictionary
x = x.to_dict(orient = "records")

In [11]:
#3 Data split
from sklearn.model_selection import train_test_split

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 10086)

In [13]:
#4 Dictionary feature extraction
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction import DictVectorizer
transfer = DictVectorizer()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [18]:
 #3 Decision tree estimator
estimator = DecisionTreeClassifier(criterion = "entropy", max_depth = 8)
estimator.fit(x_train, y_train)

#4 model assessment
#Method 1：Compare real and predicted values
y_predict = estimator.predict(x_test)
print("y_predict: \n", y_predict)
print("Comparing real and predicted values: \n", y_test == y_predict)

#Method 2：Accuracy
score = estimator.score(x_test, y_test)
print("Accuracy： \n", score)

#Visualization
export_graphviz(estimator, out_file = "titanic_tree.dot", feature_names = transfer.get_feature_names())

y_predict: 
 [0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1
 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 1
 1 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0
 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0
 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 1
 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0
 0 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0]
Comparing real and predicted values: 
 525     True
734     True
30      True
491     True
611     True
       ...  
281    False
958     True
636     True
823     True
418     True
Name: survived, Length: 329, dtype: bool
Accuracy： 
 0.8085106382978723


### Random forest prediction on survival of Titanic passengers

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

estimator = RandomForestClassifier()

#Grid search and cross validation
#preparing parameters
param_dict = {"n_estimators": [120,200,300,500,800,1200], "max_depth": [5,8,15,25,30]}
estimator = GridSearchCV(estimator, param_grid = param_dict, cv = 3)
estimator.fit(x_train, y_train)

#5 model assessment
#Method 1：compare real and predicted values
y_predict = estimator.predict(x_test)
print("y_predict: \n", y_predict)
print("Comparing real and predicted values: \n", y_test == y_predict)

#Method 2：Accuracy
score = estimator.score(x_test, y_test)
print("Accuracy is： \n", score)

# Best parameter：best_params_
print("Best parameter：\n", estimator.best_params_)
# Best score：best_score_
print("Best score：\n", estimator.best_score_)
# Best estimator：best_estimator_
print("Best estimator：\n", estimator.best_estimator_)
# Cross validation：cv_results_
print("Cross validation：\n", estimator.cv_results_)

y_predict: 
 [0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 1
 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0
 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1
 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 1 0
 0 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0]
Comparing real and predicted values: 
 525     True
734     True
30      True
491     True
611     True
       ...  
281    False
958     True
636     True
823     True
418     True
Name: survived, Length: 329, dtype: bool
Accuracy is： 
 0.8085106382978723
Best parameter：
 {'max_depth': 5, 'n_estimators': 120}
Best sc